In [1]:
# Libraries needed:
import warnings
import time
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import quantile_transform
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
import pickle                                                                      # to save trained models to disk
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# ----------------------------------- Data preparation  ----------------------------------------------
# Functions to normalize the highly-correlated feature matrix using quantile_reg  --- FUNCTION 1
def X_transform(x):
    warnings.filterwarnings(action="ignore")
    x_np= np.array(x).reshape((len(x),1))
    return quantile_transform(x_np, 
                              output_distribution='normal',
                              random_state=0).squeeze()

def X_data_transf(data,features):
    warnings.filterwarnings(action="ignore")
    data_mat= np.matrix(data.loc[:,features].apply(X_transform,axis=0))
    return np.array(data_mat)

In [4]:
# ----------------------------------- Model training and testing  ----------------------------------------------
# This is material for appendices or supp. materials in the manuscript

## --------------------------------- Pipeline for easier model fitting: ----------------------------------------
# Functions to compute prediction errors plus Bias and variance for each model         # --- FUNCTION 2

def error_functions(obs,pred):                                                           
    '''
    Compute Model BIAS and model Variance
    Complementary to MSE, MAE, R2 (agreement)
    Model BIAS and model Variance (important to assess precision/accuracy of each model)
    Usually, MSE= bias^2 + var + error; with MSE~var if estimator is unbiased
    '''
    bias= np.mean( (obs - pred) )
    var= np.mean( (pred - np.mean(pred))**2 )
    rmse= np.sqrt(mean_squared_error(obs,pred))
    mae= mean_absolute_error(obs,pred)
    r2= r2_score(obs,pred)
    
    return pd.DataFrame({"bias": [bias],"var": [var], "rmse": [rmse],'mae': [mae], 'r2': [r2]})

# Function to evaluate model performance and choose candidates                        #--- FUNCTION 3

def model_train_test (output, x_sample, test_size=0.70, phase="train", model= LinearRegression()): 
    '''
    Return model performance for a candidate ML model
    TODO:fill in more explanations
    '''
    # model set-up
    model= model
    # Sample sizes for model train/test:
    X_train, X_test, y_train, y_test= train_test_split(x_sample, output, test_size=test_size, random_state=42)
    # fit and predict
    model.fit(X_train,y_train)
    if phase == "train":
        pred= model.predict(X_train)
        err= error_functions(y_train,pred)
    else:
        pred= model.predict(X_test)
        err= error_functions(y_test,pred)
    return err

In [5]:
# ----------------------------------- Model Validation  ----------------------------------------------
# This is material for appendices or supp. materials in the manuscript
# ---------------------- Function to implement a Leave-one-out Cross Validation ----------------------
def model_validation(y_sample, x_sample, model=LinearRegression()):         # ---- FUNCTION 4
    '''
    This function implements Leave-one-out cross validation 
    It iterates over each observation, fitting the model to
    all but the observation of interest. Then, predicts
    and computes errors for that observation.
    LOOCV_error is the mean error estimate over the n-set of preds.
    '''
    
    warnings.filterwarnings(action="ignore")
    model= model
    output_length= x_sample.shape[0]
    
    #Create objects to store predictions and errors
    pred= np.empty(0)
    pred_error= np.empty(0)
    pred_sq_error= np.empty(0)
    
    # Begin the iterations: train on [n-i] and predict for [i]
    for i in range(output_length):
        
        #Prepare train and testing sets
        x_train= np.vstack(( x_sample[:i], x_sample[i+1:]))
        y_train= np.append(y_sample[:i], y_sample[i+1:]).reshape((output_length-1,1))
        
        #train in all but the i-th iteration
        model_i= model.fit(x_train, y_train)
        
        #predict for the i-th iteration
        pred_i= model_i.predict(x_sample[i].reshape(1,-1))
        
        #error for the i-th iteration (To compute BIAS)
        pred_error_i= (y_sample[i].reshape(1,-1) - pred_i)
        
        #square error for the ith iteration (To compute MSE)
        pred_sq_error_i= pred_error_i**2
        
        # Store error metrics
        pred= np.append(pred, pred_i)
        pred_error= np.append(pred_error, pred_error_i)
        pred_sq_error= np.append(pred_sq_error, pred_sq_error_i)
    
    # Compute LOOCV results
    mean_pred= np.mean(pred)
    estim_sq_error= (pred - mean_pred)**2
    r2= r2_score(y_true=y_sample, y_pred=pred)
    
    # Store all together
    model_metrics={"pred": pred,
                   "bias": pred_error,              
                   "var": estim_sq_error,
                   "mse": pred_sq_error,
                   "mae": np.abs(pred_error)}           
    print("Mean prediction estimate", mean_pred)
    print("Model R2: {}".format( round(r2, 3)))
    return pd.DataFrame(model_metrics)

In [6]:
# ----------------------------------- Model "Application"  ----------------------------------------------
# This is material for appendices or supp. materials in the manuscript
# -------- Functions to select, filter, and extract predictions at diff. N environments -----------------
# -------- Functions to resample and generate m-bootstrap estimates of 95% CIs (Biomass,N-shoot content)
# 

# Select location and n-rate combination
def select_data(location, year, n_rate_fall, n_rate_spring, data):
    y= data[
        (data["location"] == location) &
        (data["year"] == year) &
        (data["n_rate_fall"] == n_rate_fall) &
        (data["n_rate_spring"] == n_rate_spring)
    ]
    return y

#Generate m-samples out of a 4-element array(i.e. 3 our of 4 plots per N-rate treatment)
def y_boots(values, sample_size, n_samples):
    np.random.seed(0)
    indices= np.random.randint(0,4,(n_samples, sample_size))
    samples= np.empty(0)
    for index in np.arange(indices.shape[0]):
        sample_i= values[indices[index]]
        samples= np.append(samples, sample_i)
    return samples.reshape(n_samples, sample_size)

# Generate n-bootstrap estimates of mean biomass, and 95 CIs at every N-rate combination (Per location)
def bootstraps_summary(n_samples, sample_size, data, output):
    # Resample biomass
    y_bst= y_boots(values= data[output].values,
                   sample_size= sample_size,
                   n_samples= n_samples)
    
    # Mean, sd, and error bars per sample
    y_bst_means= np.apply_along_axis(np.mean, 1, y_bst)
    y_bst_sd= np.apply_along_axis(np.std, 1, y_bst) # std per sample
    y_bst_err= 1.96 * (y_bst_sd/np.sqrt(sample_size))                          
    
    y_bootstrap_dict= {
        "mean_bst": round(y_bst_means.mean(), 3),
        "std_bst": round(y_bst_sd.mean(), 3),
        "error_bst": round(y_bst_err.mean(),3)
    }
    
    y_bootstrap_df= pd.DataFrame(y_bootstrap_dict, index= np.arange(1))
    # Select row information from data to attach to each bootstrap estimate
    treatment_info= data.iloc[:1,[0,1,2,4,5]]
    # Put everything together
    y_bst_df= pd.concat([
        treatment_info.reset_index(drop=True),
        y_bootstrap_df
    ],
    axis=1)
    return y_bst_df

In [7]:
# ----------------------------------- optional functions ----------------------------------------------
# This is material for appendices or supp. materials in the manuscript
# -------- Functions to detect features of importance: Random Forest ----------------------------------

def feat_importance(X,y):
    '''
    TODO: best_model should be an instatiation of the to-be-optimized model
    x_original,cols
    '''
    best_model= RandomForestRegressor()  ## add in the optimized parameters
    best_model.fit(X,y)
    
    importances= best_model.feature_importances_
    std= np.std([tree.feature_importances_ for tree in best_model.estimators_],axis=0)
    indices= np.argsort(importances)[::-1]
    
    # Plot 
    plt.figure()
    plt.title("Feature importance Rye-biomass model, Random Forest")
    plt.bar(range(X.shape[1]), importances[indices],
           color='r', yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()
    
    #print(x_original[cols].columns[indices])